In [ ]:
import random
import numpy as np
import tensorflow as tf
from keras.models import load_model, Model
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, Callback, EarlyStopping
from keras.layers import GlobalAveragePooling2D, BatchNormalization, Input, Dense, Dropout
from keras import backend as K
from keras import optimizers
from keras.utils import to_categorical
import keras.optimizers
import pydot
import networkx as nx
from IPython.display import SVG
import glob, os
import pandas as pd
from random import shuffle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, average_precision_score, mean_absolute_error, mean_squared_error, accuracy_score
from pycausal import search as s
from pycausal.pycausal import pycausal as pc
from collections import defaultdict
from pycausal import prior as p
from causal_assurance import *
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Select your number of models
num_models = 100

# CAM-10 specified by nbest as percentage here.
nbest = int(num_models * 0.1)

# Select your number of iterations to repeat experiment
num_repeat = 100

# select your GPU Here
os.environ["CUDA_VISIBLE_DEVICES"]="" #Comment this line out if you want all GPUS 

dataset_path = '../data/hour.csv'

In [ ]:
inputs = ["Temp", "Atemp", "Humidity", "Windspeed"]
target =  ["Count"]

df = pd.read_csv(dataset_path, names = ["Temp", "Atemp", "Humidity", "Windspeed", "Count"], usecols = [10, 11, 12, 13, 16])
df = df[1:]
categoricals = [] 

temporal = [['Temp', 'Atemp', 'Humidity', 'Windspeed'], ['Count']]
prior = p.knowledge(requiredirect= [('Temp', 'Atemp'),('Windspeed', 'Atemp')],
                   addtemporal = temporal
                   )
g = examine_graph_continuous(df, prior, method = 1)
dot_str = pc.tetradGraphToDot(g)
graphs = pydot.graph_from_dot_data(dot_str)
svg_str = graphs[0].create_svg()
SVG(svg_str)

known_conx = set({})
for i in tetrad.getEdges():
    if ' o-> ' in i:
        known_conx.add((i.split(' o-> ')[0], i.split(' o-> ')[1]))
    elif ' --> ' in i:
        known_conx.add((i.split(' --> ')[0], i.split(' --> ')[1]))


prior = p.knowledge(requiredirect =  list(map(list, known_conx)),)

n_holdout = int(len(df) * 0.2)
df['Count'] = normalize(df['Count'].values.astype(int))
models = []
model_names = []

df['Temp'] = df['Temp'].values.astype(float)
df['Atemp'] = df['Atemp'].values.astype(float)
df['Windspeed'] = df['Windspeed'].values.astype(float)
df['Humidity'] = df['Humidity'].values.astype(float)
original_df = df.copy()
randomize = False
if randomize:
    layers = [256, 512, 1024, 2048, 4096]
    for i in range(num_models):
        network = []
        for j in range(3):
            network.append(layers[random.randint(0,len(layers) -1)])
        models.append(network)
        model_names.append('models/random' + str(i))
    print(models, model_names)    
else:
    model_layers = [512,256]
    for i in range(num_models):
        models.append(model_layers)
        model_names.append('models/bike' + str(i))
        
# found connections, and let's orient windspeed to humidity
prior = p.knowledge(requiredirect= [('Temp', 'Atemp'),('Windspeed', 'Atemp'),('Windspeed', 'Humidity'), ('Temp', 'Count'),('Humidity', 'Count')])
prior
print(models, model_names)
SVG(svg_str)

In [ ]:
bestMSE = []
bestCOMBO = []
for t in range(num_repeat):
    # let's split our df into two by race.  Let's see what happens if we 
    df = original_df.copy()



    holdout = int(len(df) * 0.2)
        #df_test = df[df['charges'] > 0.54].copy()
    continuous = ["Count",  "Humidity", "Atemp", "Windspeed"]
    
    
    small = random.randint(0,1)
    cont = random.randint(0, len(continuous) - 1)
    if small == 0:
        df_test = df.nsmallest(holdout, continuous[cont])
    else:
        df_test = df.nlargest(holdout, continuous[cont])
    
    print(t, small, continuous[cont])

    '''
        end_idx = len(df) - holdout
    cont = random.randint(0, len(continuous) - 1)
    start_idx = random.randint(0, end_idx)
    print(t, "Doing range:",start_idx, start_idx + holdout, "and ", continuous[cont])
    df_test = df.nlargest(len(df) - start_idx, continuous[cont]).nsmallest(holdout, continuous[cont])
    '''


    
    df.drop(df_test.index, inplace = True)
    df_test.reset_index(inplace = True)
    df.sample(frac= 1).reset_index(inplace = True) # this will shuffle and reset index

    x_test = df_test[inputs]
    y_test = df_test[target]

    causal_split = 0.2
    val_split = 0.2
    train_split = 1 - (causal_split + val_split)

    x_causal = df[inputs][-int(causal_split * len(df)) :]
    y_causal = df[target][-int(causal_split * len(df)) :]

    x_val = df[inputs][int(train_split * len(df)):-int(causal_split * len(df))]
    y_val = df[target][int(train_split * len(df)):-int(causal_split * len(df))]

    x_train = df[inputs][:int(train_split * len(df))]
    y_train = df[target][:int(train_split * len(df))]
    len(x_causal), len(y_causal), len(x_val), len(y_val), len(x_train), len(y_train)



    x_test_NN = x_test
    x_causal_NN = x_causal
    x_val_NN = x_val
    x_train_NN = x_train

    verbosity = 0


    for idx, model_name in enumerate(model_names):
        #print(model_name)

        if type(models[idx]) is list:
            #clear session
            keras.backend.clear_session() 
            #get model according to specification
            model = get_model(models[idx], [0.2] * len(models), np.shape(x_train_NN)[1])
            callbacks = [ModelCheckpoint(model_name, verbose= verbosity, monitor='val_loss',save_best_only=True), 
                         EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose= verbosity, mode='auto')]
            model.compile(optimizer = optimizers.SGD(lr = 0.001, momentum = 0.9, ), loss='mean_squared_error', metrics = ['mse'])
            model.fit(x_train_NN, y_train, epochs = 20, validation_data = (x_val_NN, y_val), callbacks = callbacks, batch_size = 32, verbose = verbosity)
        else:
            models[idx].fit(X,y)




    generalization = []
    metrics = []
    proposed = []
    x_causal.reset_index(drop = True, inplace = True)

    for idx, model_name in enumerate(model_names):
        #print(model_name)
        if type(models[idx]) is list:
            keras.backend.clear_session()
            model = load_model(model_name)
        else:
            model = models[idx]

        y_pred = model.predict(x_test_NN)
        generalization.append(mean_squared_error(y_pred, y_test))

        #### CHECK FOR CAUSAL METRIC HERE
        y_causal_pred = model.predict(x_causal_NN)
        causal_targets = pd.DataFrame(y_causal_pred, columns = target)
        causal_targets.reset_index(drop=True, inplace = True)
        causal_df = x_causal.join(causal_targets)

        metrics.append(mean_squared_error(y_causal_pred, y_causal))

        ll_pred = get_ll_continuous(causal_df, prior)

        proposed.append(ll_pred)

    total = normalize(metrics) + normalize(proposed)
    final = pd.DataFrame(np.stack((metrics, proposed, total, normalize(generalization)), axis = 1), columns = ['metrics', 'proposed', 'combined', 'generalization'])
    print("MSE = ", np.mean(final.nsmallest(nbest, 'metrics')['generalization']))
    print("COMB = ",np.mean(final.nsmallest(nbest, 'combined')['generalization']))
    bestMSE.append(final.nsmallest(nbest, 'metrics')['generalization'].values)
    bestCOMBO.append(final.nsmallest(nbest, 'combined')['generalization'].values)
    

np.mean(bestMSE), np.mean(bestCOMBO), np.std(bestMSE), np.std(bestCOMBO)

In [ ]:
import seaborn as sns
val1 = []
for each in bestMSE:
    val1.append(np.mean(each))
val2 = []
for each in bestCOMBO:
    val2.append(np.mean(each))

val = []
for x, y in zip(val1, val2):
    val.append([x, y])
    
fig, ax = plt.subplots()
fig.set_size_inches(2.5,3)
df = pd.DataFrame(val, columns = ['MSE', 'Proposed'])
ax = sns.boxplot(ax = ax, data=df, palette="Set2")
fig.savefig('kaggle-bikeshare.pdf')
d = dict()
d['bestMSE'] = bestMSE
d['bestCOMBO'] = bestCOMBO


import pickle

with open('BikeShare.pkl', 'wb') as handle:
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)
